In [1]:
import numpy as np
import cv2
import os

In [2]:
def distance(v1,v2):
    # Eucledian
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist =[]

    for i in range (train.shape[0]):
        # Get the vector and label
        ix = train[1, :-1] # rest of the column for feature part
        iy = train[1, -1] #last column for label part
        # Compute the dist from the test point
        d = distance(test, ix)  
        dist.append((d,iy))   #dist and label

    # Sort based on distance and get top k
    dk = sorted (dist, key=lambda x: x[0])[:k]
    # Retrieve only the labes
    labels = np.array(dk)[:, -1] # converting into numpy array to perform more operation

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])

    return output[0][index] 

In [3]:
cap = cv2.VideoCapture(0)  

face_cascade_name = cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier()
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print("Error loading xml file")
    exit(0)
skip = 0
dataset_path = 'D:/IIEST clg/Project/Personal/Face_recognition/'

face_data =[]
labels =[]

class_id = 0 #labels for the given file
names = {} # a dictionary to create a mapping btw id - name

#Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        #Create a mapping between class_id and name
        names[class_id] = fx[:-4]
        
        print("Loaded"+fx)
        
        data_item = np.load (dataset_path+fx)
        face_data.append(data_item)

        #Create labels for the class
        target = class_id*np.ones((data_item.shape[0],)) 
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1) #aopending 1 coloumn
print(trainset.shape)

# Testing

while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h = face
        
        #Get the face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        #Predicted Label (out)
        out = knn(trainset, face_section.flatten())

        #Display on the scrn the name & rectangle arount it
        pred_name = names[int(out)]
        
        #Adding text to images
        cv2.putText(frame, pred_name,(x,y-10), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA) 
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces", frame)

    key_pressed = cv2.waitKey(1) & 0xFF #a 32 bit int is multiplied by a 8 bit int to get a 8 bit int
    if key_pressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loadedpranjal.npy
Loadedrajib.npy
(22, 30000)
(22, 1)
(22, 30001)
